# https://bigdata-tools.com/imbalance-data/ (いくつか修正版)

In [1]:
%%HTML
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>

In [2]:
import pandas as pd
from sklearn.datasets import make_classification

pd.set_option("display.max_columns", None)

In [3]:
#　サンプル数10000、特徴量10のデータを人工的に作成。陰性クラス(0)：陽性クラス(1)＝99.9：0.1

data_base = make_classification(
    n_samples = 10000, n_features = 10, n_informative = 2, n_redundant = 0, 
    n_repeated = 0, n_classes = 2, n_clusters_per_class = 2, weights = [0.999, 0.001], 
    flip_y = 0, class_sep = 1.0, hypercube = True, shift = 0.0, 
    scale = 1.0, shuffle = True, random_state = 100)

In [4]:
df = pd.DataFrame(data_base[0], columns = ['特徴量1', '特徴量2', '特徴量3', '特徴量4', '特徴量5', '特徴量6', '特徴量7', '特徴量8', '特徴量9', '特徴量10'])
df['クラス'] = data_base[1]

## 不均衡データでロジスティック回帰

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# データを学習用と検証用に分ける
x = df.iloc[:, 0:10]
y = df['クラス']
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 100)

# 分類モデル作成
clf = LogisticRegression()
clf.fit(x_train, y_train)

# 作成したモデルで、テストデータを予測値
y_pred = clf.predict(x_test)

In [6]:
# 混同行列を作る
cm = confusion_matrix(y_test, y_pred)
cm

array([[2497,    1],
       [   2,    0]])

In [7]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).flatten()
print(dict(zip(['tn','fp','fn','tp'], (tn, fp, fn, tp))))
#精度を求める
print("精度：{:.4f}".format(accuracy_score(y_test, y_pred)))
print("適合率：{:.4f}".format(tp/(tp+fp)))
print("再現率：{:.4f}".format(tp/(tp+fn)))

{'tn': 2497, 'fp': 1, 'fn': 2, 'tp': 0}
精度：0.9988
適合率：0.0000
再現率：0.0000


# アンダーサンプリングで学習用データ作成 (RandomUnderSampler)

In [8]:
from imblearn.under_sampling import RandomUnderSampler

# クラス1の数を保存
count_train_class_one = y_train.sum()
print('クラス1のサンプル数:{}'.format(count_train_class_one)) #クラス1のサンプル数表示

# クラス1が全体の10％になるまでクラス0を減らす
under = RandomUnderSampler(sampling_strategy=0.11, random_state=100)

# 学習用データに反映
x_train_under, y_train_under = under.fit_resample(x_train, y_train)

print(x_train_under.shape) #学習用データのサンプル数確認

クラス1のサンプル数:8
(80, 10)


In [9]:
clf.fit(x_train_under, y_train_under)

# 作成したモデルで、テストデータを予測値
y_under_pred = clf.predict(x_test)

In [10]:
# 混同行列を作る
cm = confusion_matrix(y_test, y_under_pred)
cm

array([[2471,   27],
       [   1,    1]])

In [11]:
tn, fp, fn, tp = cm.flatten()
print(dict(zip(['tn','fp','fn','tp'], (tn, fp, fn, tp))))
#精度、適合率、再現率を求める
print("精度：{:.4f}".format(accuracy_score(y_test, y_under_pred)))
print("適合率：{:.4f}".format(tp/(tp+fp)))
print("再現率：{:.4f}".format(tp/(tp+fn)))

{'tn': 2471, 'fp': 27, 'fn': 1, 'tp': 1}
精度：0.9888
適合率：0.0357
再現率：0.5000


# SMOTEで学習用データ作成 (SMOTE)

In [12]:
from imblearn.over_sampling import SMOTE

# クラス1の数を保存
print('SMOTE前のクラス0のサンプル数:{}'.format(y_train[y_train==0].count())) #クラス0のサンプル数表示
print('SMOTE前のクラス1のサンプル数:{}'.format(y_train[y_train==1].count())) #クラス1のサンプル数表示

# クラス0：クラス1=9:1になるまでクラス1を増やす
smote = SMOTE(sampling_strategy = 0.1, random_state=100)

# 学習用データに反映
x_train_smote, y_train_smote = smote.fit_resample(x_train, y_train)

print(x_train_smote.shape) #学習用データのサンプル数確認
print("SMOTE後のクラス0のサンプル数:{}".format(y_train_smote[y_train_smote==0].count())) #クラスのサンプル数
print("SMOTE後のクラス1のサンプル数:{}".format(y_train_smote[y_train_smote==1].count())) #クラス1のサンプル数

SMOTE前のクラス0のサンプル数:7492
SMOTE前のクラス1のサンプル数:8
(8241, 10)
SMOTE後のクラス0のサンプル数:7492
SMOTE後のクラス1のサンプル数:749


In [13]:
#SMOTEでロジスティック回帰

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# 分類モデル作成
clf_smote = LogisticRegression()
clf_smote.fit(x_train_smote, y_train_smote)

# 作成したモデルで、テストデータを予測値
y_smote_pred = clf_smote.predict(x_test)

In [14]:
# 混同行列を作る
cm = confusion_matrix(y_test, y_smote_pred)
cm

array([[2477,   21],
       [   0,    2]])

In [15]:
tn, fp, fn, tp = cm.flatten()
print(dict(zip(['tn','fp','fn','tp'], (tn, fp, fn, tp))))
#精度、適合率、再現率を求める
print("精度：{:.4f}".format(accuracy_score(y_test, y_smote_pred)))
print("適合率：{:.4f}".format(tp/(tp+fp)))
print("再現率：{:.4f}".format(tp/(tp+fn)))

{'tn': 2477, 'fp': 21, 'fn': 0, 'tp': 2}
精度：0.9916
適合率：0.0870
再現率：1.0000
